# Extended Figure 8
Spatial transcriptomic analysis of IEC in response to acute nociceptor activation.

In [ ]:
from pathlib import Path
import sys
import os
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = (4,4)
plt.rcParams["figure.dpi"] = 150

In [ ]:
sys.path.append(str(Path.cwd().resolve().parents[1]))

from config.paths import BASE_DIR

input_dir = BASE_DIR / "data/h5ad/export_11"
output_dir = BASE_DIR / "figures/ExtdFig_8"

output_dir.mkdir(parents=True, exist_ok=True)

input_file = input_dir / "adata-v3-annotations.h5ad"

In [ ]:
sc.settings.figdir = output_dir

In [ ]:
adata = sc.read_h5ad(input_file)
adata

In [ ]:
adata.obs["Class"] = adata.obs["Class"].replace({"Epithelial": "IEC"})

### ExtdFig 8c

In [ ]:
groups_to_plot = ["IEC", "Immune", "Stromal", "Neural"]

custom_colors = {
    "IEC": "#fc8d62",
    "Immune": "#9c89b8",
    "Stromal": "#66c2a5",
    "Neural": "#e78ac3",
}
color_list = [custom_colors[group] for group in groups_to_plot]

In [ ]:
adata_subset = adata[adata.obs["Class"].isin(groups_to_plot)].copy()

adata_subset.obs["Class"] = pd.Categorical(
    adata_subset.obs["Class"], categories=groups_to_plot, ordered=True
)

In [ ]:
sc.pl.umap(
    adata_subset,
    color="Class",
    palette=color_list,
    frameon=False,
    title="",
    legend_fontsize=14,
    save = '_ExtdFig8c.pdf'
)

In [ ]:
plt.rcdefaults()
plt.rcParams["figure.dpi"] = 150

### ExtdFig 8d

In [ ]:
edata = adata[adata.obs["Class"] == "IEC"].copy()

In [ ]:
sc.tl.rank_genes_groups(edata, groupby="cell_type", method="wilcoxon")

In [ ]:
markers = sc.get.rank_genes_groups_df(edata, group=None)
markers

In [ ]:
log2fc_threshold = 2
pval_threshold = 0.05
markers_filtered = markers[
    (markers["logfoldchanges"] > log2fc_threshold)
    & (markers["pvals_adj"] < pval_threshold)
].copy()

#### Extract top 5 genes for each group

In [ ]:
top_genes = (
    markers_filtered.sort_values(
        by=["group", "logfoldchanges"], ascending=[True, False]
    )  # Sort by group and log2FC
    .groupby("group")["names"]  
    .head(5)  
    .tolist()  
)

In [ ]:
top = ["Epcam"] + top_genes

In [ ]:
plt.rcParams["font.size"] = 15

In [ ]:
sc.pl.dotplot(
    edata,
    var_names=top,
    groupby="cell_type",
    title="",
    save = '_ExtdFig8d.pdf'
)

### ExtdFig 8e

In [ ]:
input_dir = BASE_DIR / "data/h5ad/export_10"
input_file = input_dir / "resolvi-corrected-prepped.h5ad"


edata = sc.read_h5ad(input_dir / "iec-subset-resolvi-cc-v2.h5ad") # epithelial subset with cellcharter annotations

In [ ]:
def plot_spatial_highlight_zoom(
    adata, basis, label_key, fov=None, size=50, palette=None, vmin=-0.5, vmax=0.5
):
    if palette is None:
        palette = ["#EF5703", "#E3B710", "#91BAB6"]  
    group_order = ["Late", "Early", "Stem/Progenitor"]  

    color_dict = {group: color for group, color in zip(group_order, palette)}

    for label in adata.obs[label_key].astype(str).unique():
        if label not in color_dict:
            color_dict[label] = "lightgray" 

    # Create the plot
    fig, ax = plt.subplots(figsize=(6, 2))
    sc.pl.embedding(
        adata,
        basis=basis,
        color=label_key,
        palette=color_dict,
        ax=ax,
        show=False,
        size=size,
        frameon=False,
        title="",
    )

    # Apply zooming if FOV is provided
    if fov:
        xmin, xmax, ymin, ymax = fov
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)

    plt.show()
    return fig

In [ ]:
palette = [
    "black",  
    "#4A6FE3",  
    "#C72C35", 
]

In [ ]:
zone_mapping = {0: "Stem/Progenitor", 1: "Early", 2: "Late"}
edata.obs["cellcharter_zones"] = edata.obs["epithelial_cc_3"].map(zone_mapping)

all_neighborhoods = edata.obs["cellcharter_zones"].astype(str).unique().tolist()
edata.obs["cellcharter_zones"] = edata.obs["cellcharter_zones"].astype(str)

In [ ]:
cdata = edata[edata.obs["sample_id"] == "TIS09472_Control"].copy()

In [ ]:
fig = plot_spatial_highlight_zoom(
    cdata,
    basis="spatial",
    label_key="cellcharter_zones",
    fov=(2916, 4321, 5150, 5700),
    size=50,
    palette=palette,
    vmin=0,
    vmax=1,
)

fig.tight_layout()
fig.savefig(f"{output_dir}/ExtdFig8e.pdf", dpi=300, bbox_inches="tight")

### ExtdFig 8f

In [ ]:
heatmap_data = pd.crosstab(edata.obs["cell_type"], edata.obs["epithelial_cc_3"])
heatmap_data_norm = heatmap_data.div(
    heatmap_data.sum(axis=0), axis=1
)  # normalize by column to get cell type proportions

In [ ]:
plt.figure(figsize=(4, 4))
sns.heatmap(heatmap_data_norm, cmap="Reds", annot=True, fmt=".2f")
plt.title("")
plt.xlabel("IEC Composition per Spatial Zone")
plt.ylabel("")
plt.tight_layout()
save_path = os.path.join(output_dir, "ExtdFig8f.pdf")
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

### ExtdFig 8g

In [ ]:
def plot_spatial_highlight_zoom(
    adata, basis, label_key, fov=None, size=50, palette="viridis", vmin=-0.5, vmax=0.5
):

    zissou = [
        "#3A9AB2",
        "#6FB2C1",
        "#91BAB6",
        "#A5C2A3",
        "#BDC881",
        "#DCCB4E",
        "#E3B710",
        "#E79805",
        "#EC7A05",
        "#EF5703",
        "#F11B00",
    ]
    zissou_colormap = clr.LinearSegmentedColormap.from_list("Zissou", zissou)

    # Extract coordinates and continuous values
    x_coords = adata.obsm[basis][:, 0]
    y_coords = adata.obsm[basis][:, 1]
    continuous_values = adata.obs[label_key].values

    # Clip and normalize
    continuous_values_clipped = np.clip(continuous_values, vmin, vmax)
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Select colormap
    cmap = zissou_colormap if palette == "zissou" else cm.get_cmap(palette)

    # Color mapping
    colors = cmap(norm(continuous_values_clipped))

    fig, ax = plt.subplots(figsize=(6, 2))
    ax.scatter(x_coords, y_coords, c=colors, s=size, edgecolors="none", alpha=0.85)

    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, fraction=0.02, pad=0.04)
    cbar.set_label("", fontsize=12)  

    if fov:
        xmin, xmax, ymin, ymax = fov
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    for spine in ax.spines.values():
        spine.set_visible(False)

    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title("")

    plt.tight_layout()
    plt.show()

    return fig

In [ ]:
cdata = edata[edata.obs["sample_id"] == "TIS09472_Control"].copy()

In [ ]:
fig = plot_spatial_highlight_zoom(
    adata=cdata,
    basis="spatial",
    label_key="crypt_villus_axis_scaled",
    fov=(2916, 4321, 5200, 5700),
    size=10,
    palette="zissou",
    vmin=0,
    vmax=1,
)

fig.savefig(os.path.join(output_dir, "ExtdFig8g.pdf"), bbox_inches="tight")

### ExtdFig 8h

In [ ]:
df = edata.obs[["crypt_villus_axis_scaled", "cellcharter_zones"]].dropna()
cluster_colors = {"Late": "black", "Early": "#4A6FE3", "Stem/Progenitor": "#C72C35"}
clusters = sorted(df["cellcharter_zones"].unique())

In [ ]:
plt.figure(figsize=(4, 3))

sns.kdeplot(
    data=df,
    x="crypt_villus_axis_scaled",
    hue="cellcharter_zones",
    common_norm=False,
    fill=True,
    alpha=0.5,
    linewidth=0,
    palette=cluster_colors,
    legend=False,
)

for cluster in clusters:
    group = df[df["cellcharter_zones"] == cluster]
    q25 = np.percentile(group["crypt_villus_axis_scaled"], 25)
    q75 = np.percentile(group["crypt_villus_axis_scaled"], 75)

    plt.axvline(x=q25, color=cluster_colors[cluster], linestyle="--", linewidth=1.5)
    plt.axvline(x=q75, color=cluster_colors[cluster], linestyle="--", linewidth=1.5)

legend_elements = [
    Patch(facecolor=cluster_colors[c], label=c, alpha=0.8) for c in clusters
]
plt.legend(
    handles=legend_elements, title="Cluster", bbox_to_anchor=(1.05, 1), loc="upper left"
)

plt.xlabel("Crypt-Villus Axis")
plt.ylabel("Cell Density")
plt.title("Cell Positions by Zone")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "ExtdFig8h.pdf"))
plt.show()